In [1]:
!pip install joblib
"""!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4"""
!pip install transformers

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Importing Libs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
#from huggingface_hub import hf_hub_download
#from llama_cpp import Llama
import re
import torch
import torch.nn as nn
import shutil
import sys
from transformers import BertTokenizer,BertModel
import joblib
from collections import Counter

In [4]:
dataset = pd.read_csv("/content/tabular_dataset.csv")

In [5]:
tabular_columns = dataset.columns
tabular_columns = tabular_columns.drop('prognosis')

adjusted_tabular_columns = [term.replace('_', ' ') for term in tabular_columns]
print(adjusted_tabular_columns)

['itching', 'skin rash', 'nodal skin eruptions', 'continuous sneezing', 'shivering', 'chills', 'joint pain', 'stomach pain', 'acidity', 'ulcers on tongue', 'muscle wasting', 'vomiting', 'burning micturition', 'spotting urination', 'fatigue', 'weight gain', 'anxiety', 'cold hands and feets', 'mood swings', 'weight loss', 'restlessness', 'lethargy', 'patches in throat', 'irregular sugar level', 'cough', 'high fever', 'sunken eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish skin', 'dark urine', 'nausea', 'loss of appetite', 'pain behind the eyes', 'back pain', 'constipation', 'abdominal pain', 'diarrhoea', 'mild fever', 'yellow urine', 'yellowing of eyes', 'acute liver failure', 'fluid overload', 'swelling of stomach', 'swelled lymph nodes', 'malaise', 'blurred and distorted vision', 'phlegm', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'congestion', 'chest pain', 'weakness in limbs', 'fast heart rate', 'pain during bowe

In [6]:
#### LOADING MODELS ####

mlp_path = '/content/mlp.joblib'
mlp = joblib.load(mlp_path)

SVC_path = '/content/SVC.joblib'
SVC = joblib.load(SVC_path)

Random_path = '/content/Random.joblib'
Random = joblib.load(Random_path)

GaussianNB_path = '/content/GaussianNB_gridsearch.joblib'
GaussianNB = joblib.load(GaussianNB_path)

Decision_path = '/content/Decision Tree.joblib'
Decision = joblib.load(Decision_path)

kNearest_path = '/content/k-Nearest.joblib'
kNearest = joblib.load(kNearest_path)

In [ ]:
#### LLAMA TRAINING####

"""model_name = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

model_path = hf_hub_download(repo_id=model_name, filename=model_basename)
lcpp_llm = None
lcpp_llm = Llama(model_path=model_path,n_threads=2, n_batch=512, n_gpu_layers=32)
"""

'model_name = "TheBloke/Llama-2-13B-chat-GGML"\nmodel_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"\n\nmodel_path = hf_hub_download(repo_id=model_name, filename=model_basename)\nlcpp_llm = None\nlcpp_llm = Llama(model_path=model_path,n_threads=2, n_batch=512, n_gpu_layers=32)\n'

In [7]:
#### BERT TRAINING ####
dataset_path = "/content/text_dataonly.csv"
df = pd.read_csv(dataset_path)
encoded_labels = pd.get_dummies(df['label'])
df = pd.concat([df['text'], encoded_labels], axis=1)
df.columns = ['text'] + list(encoded_labels.columns)
target = encoded_labels.columns
max_len = 256
train_batch_size=32
val_batch_size=32
epochs=2
lr = 1e-05
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
class diseaseDataset(torch.utils.data.Dataset):
  #constructor
  def __init__(self, df, tokenizer, max_len):
    self.df=df
    self.tokenizer=tokenizer
    self.max_len=max_len
    self.title = self.df["text"]
    self.targets = self.df[target].values

  def __len__(self):
    return len(self.title)

  #so no garabge
  def __getitem__(self,index):
    title = str(self.title[index])
    title = " ".join(title.split())

    inputs = self.tokenizer.encode_plus(
        title,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        padding="max_length",
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return {
      'input_ids': inputs['input_ids'].flatten(),
      'attention_mask': inputs['attention_mask'].flatten(),
      'token_type_ids':inputs['token_type_ids'].flatten(),
      'targets':torch.FloatTensor(self.targets[index])
    }
train_df, test_df= train_test_split(df,test_size=0.1,random_state=2)
train_df=train_df.reset_index(drop=True)
test_df=test_df.reset_index(drop=True)
train_df, val_df= train_test_split(train_df,test_size=0.2,random_state=2)
train_df=train_df.reset_index(drop=True)
val_df=val_df.reset_index(drop=True)
train_dataset = diseaseDataset(train_df, tokenizer, max_len)
val_dataset = diseaseDataset(val_df, tokenizer, max_len)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=train_batch_size,
    num_workers=0
)
val_data_loader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle=False,
    batch_size=val_batch_size,
    num_workers=0
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def load_checkpoint(checkpoint_path, model, optimizer):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_checkpoint(state, is_best, checkpoint_path, best_model_path):
    path = checkpoint_path
    torch.save(state, path)
    if is_best:
        best_path = best_model_path
        shutil.copyfile(path, best_path)
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True) #1st layer
        self.dropout = torch.nn.Dropout(0.1) #dropout layer
        self.linear = torch.nn.Linear(768, 12) #bert_op = 768 layers

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)
def loss_func(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params= model.parameters(), lr=lr)
def train_model(n_epochs, train_loader, val_loader, model, opt, ckp_path, best_path):
  val_loss_min= np.Inf
  for epoch in range(1,epochs+1):
    train_loss=0
    val_loss=0
    model.train()
    for idx, batch in enumerate(train_loader):
      ip_ids=batch["input_ids"].to(device, dtype=torch.long)
      attention_mask=batch["attention_mask"].to(device, dtype=torch.long)
      token_type_ids=batch["token_type_ids"].to(device, dtype=torch.long)
      targets = batch["targets"].to(device, dtype=torch.float)
      op = model(ip_ids, attention_mask, token_type_ids)
      opt.zero_grad()
      loss = loss_func(op, targets)
      opt.zero_grad()
      loss.backward()
      opt.step()
      train_loss=train_loss+((1/(idx+1))*(loss.item()-train_loss))
    model.eval()
    with torch.no_grad():
      for batch_idx, data in enumerate(val_loader,0):
        ip_ids=batch["input_ids"].to(device, dtype=torch.long)
        attention_mask=batch["attention_mask"].to(device, dtype=torch.long)
        token_type_ids=batch["token_type_ids"].to(device, dtype=torch.long)
        targets = batch["targets"].to(device, dtype=torch.float)
        op = model(ip_ids, attention_mask, token_type_ids)
        loss = loss_func(op, targets)
        val_loss=val_loss+((1/(idx+1))*(loss.item()-val_loss))

      train_loss = train_loss/len(train_loader)
      val_loss = val_loss/len(val_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, train_loss,val_loss))
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': val_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
      save_checkpoint(checkpoint, False, ckp_path, best_path)
  return model
trained_model = train_model(epochs, train_data_loader, val_data_loader,model,optimizer,"/current_ckpt","/best_ckpt")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch: 1 	Avgerage Training Loss: 0.010389 	Average Validation Loss: 0.006712
Epoch: 2 	Avgerage Training Loss: 0.006812 	Average Validation Loss: 0.005358


In [8]:
#Disease and medicine mapping

df = pd.read_csv('drugs_for_common_treatments.csv')
tabular_columns = df.columns
medical_conditions = df['medical_condition']
drug_name = df['drug_name']
dic = {'Bronchial Asthma': [], 'Migraine': [], 'Jaundice': [], 'Malaria': [], 'Dengue': [], 'Typhoid': [], 'Common Cold': [], 'Pneumonia': [], 'Arthritis': [], 'Acne': [], 'Psoriasis': [], 'Impetigo': []}

for i in range(len(medical_conditions)):
    if medical_conditions[i] in dic:
        dic[medical_conditions[i]].append(drug_name[i])
print(dic)

{'Bronchial Asthma': [], 'Migraine': ['sumatriptan', 'Imitrex', 'Zomig', 'Cambia', 'Nurtec ODT', 'zolmitriptan', 'rizatriptan', 'Maxalt', 'Reyvow', 'Elyxyb', 'lasmiditan', 'rimegepant', 'Tosymra', 'gabapentin', 'Relpax', 'Excedrin Migraine', 'Amerge', 'cyclobenzaprine', 'Fiorinal', 'Treximet', 'Maxalt-MLT', 'diclofenac', 'Ubrelvy', 'naratriptan', 'aspirin / butalbital / caffeine', 'Frova', 'Botox', 'eletriptan', 'Excedrin', 'Reglan', 'metoclopramide', 'almotriptan', 'naproxen / sumatriptan', 'venlafaxine', 'acetaminophen / aspirin / caffeine', 'cyproheptadine', 'Imitrex Statdose', 'Methergine', 'Migranal', 'acetaminophen / aspirin / caffeine / salicylamide', 'Cafergot', 'dihydroergotamine', 'frovatriptan', 'memantine', 'Migergot', 'orphenadrine', 'Zembrace SymTouch', 'Zomig-ZMT', 'caffeine / ergotamine', 'celecoxib', 'D.H.E. 45', 'Ergomar', 'ergotamine', 'methylergonovine', 'onabotulinumtoxinA', 'Onzetra Xsail', 'Painaid', 'prochlorperazine', 'Vanquish', 'Zomig Nasal Spray', 'Anacin Ad

In [9]:
for i in dic:
    if dic[i] == []:
        if i == 'Common Cold':
            dic[i] = ['Antihistamines','Cough suppressants','Decongestants']
        elif i == 'Bronchial Asthma':
            dic[i] = ['Antileukotriene','Inhaled corticosteroids','Albuterol']
        elif i == 'Jaundice':
            dic[i] = ['Cholestyramine','Colestipol']
        elif i == 'Malaria':
            dic[i] = ['Atovaquone/Proguanil','Chloroquine','Doxycycline']
        elif i == 'Dengue':
            dic[i] = ['No specific antiviral agents exist for dengue. Please contact your consultant.']
        elif i == 'Typhoid':
            dic[i] = ['Fluoroquinolones','Cephalosporins']
        elif i == 'Arthritis':
            dic[i] = ['Corticosteroids','Diclofenac','Ibuprofen']
        elif i == 'Impetigo':
            dic[i] = ['Cephalexin','Clindamycin','Trimethoprim']
    else:
        dic[i] = dic[i][:3]

print(dic)
print(len(dic))

medic_dic = dic

{'Bronchial Asthma': ['Antileukotriene', 'Inhaled corticosteroids', 'Albuterol'], 'Migraine': ['sumatriptan', 'Imitrex', 'Zomig'], 'Jaundice': ['Cholestyramine', 'Colestipol'], 'Malaria': ['Atovaquone/Proguanil', 'Chloroquine', 'Doxycycline'], 'Dengue': ['No specific antiviral agents exist for dengue. Please contact your consultant.'], 'Typhoid': ['Fluoroquinolones', 'Cephalosporins'], 'Common Cold': ['Antihistamines', 'Cough suppressants', 'Decongestants'], 'Pneumonia': ['levofloxacin', 'clarithromycin', 'ceftriaxone'], 'Arthritis': ['Corticosteroids', 'Diclofenac', 'Ibuprofen'], 'Acne': ['doxycycline', 'spironolactone', 'minocycline'], 'Psoriasis': ['clobetasol', 'methotrexate', 'Humira'], 'Impetigo': ['Cephalexin', 'Clindamycin', 'Trimethoprim']}
12


In [10]:
def bmm_helper(symptom):
  pred_disease = []

  symptom_lower = symptom.lower()

  vector = [1 if any(term in symptom_lower for term in column.split()) else 0 for column in adjusted_tabular_columns]
  vector = np.array(vector)

  vector_array = np.array(vector).reshape(1, -1)
  #print(vector_array)

  #GaussianNB
  y_pred = GaussianNB.predict(vector_array)
  pred_disease.append(y_pred[0])

  #MLP
  pred = mlp.predict(vector_array)
  pred_disease.append(pred[0])

  #Decision Tree
  pred = Decision.predict(vector_array)
  pred_disease.append(pred[0])

  #Random Forest
  pred_rf = Random.predict(vector_array)
  pred_disease.append(pred_rf[0])

  #SVC
  pred_svc = SVC.predict(vector_array)
  pred_disease.append(pred_svc[0])

  #k-Nearest
  pred_kNearest = kNearest.predict(vector_array)
  pred_disease.append(pred_kNearest[0])

  """#LLAMA PREDICTION
  prompt = "Please tell the disease based on the following symptoms:"
  symptoms = symptom
  prompt_template=f'''SYSTEM: Based on the symptoms described below, respond ONLY with the name of the most probable disease and enclose it in <disease> answer </disease>. Do not include any explanations, sentences, or additional text. Write your response on a new line immediately following this prompt.
  USER: {prompt}{symptoms}
  ASSISTANT:
  '''
  response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.4, top_p=0.9,repeat_penalty=1.5, top_k=100, echo=True)
  op = (response["choices"][0]["text"]).strip().split("\n")[-1]
  match = re.search(r'<disease>(.*?)</disease>', op)
  if match:
      disease_name = match.group(1).split()[0]
      print(disease_name)
  else:
      print("Disease name not found in the response.")
  pred_disease.append(disease_name)"""

  #BERT PREDICTION
  example = symptom
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
      bert_op=train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))]
      pred_disease.append(bert_op)
  counts = Counter(pred_disease)
  max_occurrence = max(counts.values())
  max_values = [value for value, count in counts.items() if count == max_occurrence]
  if len(max_values) > 1:
    return bert_op
  else:
    return max_values[0]

In [11]:
symptom_running = ""

In [ ]:
symptom = "I have high fever"

In [13]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=4ec723c8dc93f0f10d5bb7aca8f3b5e74d38d5c0cd95f3cf5f4920151ec54c07
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [14]:
import anvil.server

In [15]:
anvil.server.connect("server_GRQWP2QZSHZKRZ5J3ZOAFNHM-GDT5CWK6NEUHTFRA")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [16]:
clear = False

In [17]:
@anvil.server.callable
def be_my_memory(symptom):
  global symptom_running
  global medic_dic
  global clear

  medi = set()

  symptom_lower = symptom.lower()

  vector = [1 if any(term in symptom_lower for term in column.split()) else 0 for column in adjusted_tabular_columns]
  vector = np.array(vector)

  vector_array = np.array(vector).reshape(1, -1)
  #print(vector_array)

  #count number of 1s
  ones = np.count_nonzero(vector_array == 1)
  one=True
  if ones == 0:
    one=False
  elif ones == 1:
    one=True
  else:
    if clear == True:
      symptom_running = ""
      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)
      val = "Based on current symptoms, you might have " + current_diag + ". But we suggest you to consult a doctor."

      for i in medic_dic[current_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string

    if(symptom_running==""):
      symptom_running=symptom
      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)
      val = "Based on current symptoms, you might have " + current_diag + ". But we suggest you to consult a doctor."

      for i in medic_dic[current_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string

    else:
      symptom_running = symptom_running + " " + symptom

      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)

      #based on concatenated data
      concat_diag = bmm_helper(symptom_running)

      val = "Based on current symptoms, you might have " + current_diag + " and considering your history, you could have " + concat_diag + ". But we suggest you to consult a doctor."

      for i in medic_dic[current_diag]:
        medi.add(i)

      for i in medic_dic[concat_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string


  if one==False:
    val = "Based on current domain of dieseases, we cannot predict any disease confidentaly. But we highly suggest you to consult a doctor."

  if ones == 1 and one==True:
    # current_diag = bmm_helper(symptom)
    # val = "We predict you have" + current_diag + " but not many symptons matched. So we highly suggest you to consult a doctor."

    if clear == True:
      symptom_running = ""
      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)
      # val = "Based on current symptoms, you might have " + current_diag + ". But we suggest you to consult a doctor."
      val = "We predict you have " + current_diag + " but not many symptons matched. So we highly suggest you to consult a doctor."

      for i in medic_dic[concat_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string

    if(symptom_running==""):
      symptom_running = symptom
      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)
      # val = "Based on current symptoms, you might have " + current_diag + ". But we suggest you to consult a doctor."
      val = "We predict you have" + current_diag + " but not many symptons matched. So we highly suggest you to consult a doctor."

      for i in medic_dic[concat_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string

    else:
      symptom_running = symptom_running + " " + symptom

      #based on current data
      current_diag = bmm_helper(symptom)
      #print(current_diag)

      #based on concatenated data
      concat_diag = bmm_helper(symptom_running)

      # val = "Based on current symptoms, you might have " + current_diag + " and considering your history, you could have " + concat_diag + ". But we suggest you to consult a doctor."
      val = "Based on current symptoms, you might have " + current_diag + " and considering your history, you could have " + concat_diag + ". But we highly suggest you to consult a doctor as not many symptoms matched."

      for i in medic_dic[current_diag]:
        medi.add(i)

      for i in medic_dic[concat_diag]:
        medi.add(i)

      medi_string = ", ".join(medi)
      val += " Recommended medicines: " + medi_string

  return val

In [18]:
val = be_my_memory(symptom)
print(val)
#print(symptom_running)

NameError: name 'symptom' is not defined

In [ ]:
lst = be_my_memory(symptom)
print(lst)

Migraine


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifie

In [21]:
anvil.server.wait_forever()

KeyboardInterrupt: 